# Imports

In [11]:
import requests
import pandas as pd
import os
from functools import reduce
import geopandas as gpd


# Fetch 4 neighbourhoods

In [27]:
equiv = pd.read_csv("2020 Census Tracts to Chicago Community Area Equivalency File - Sheet1.csv")


print(equiv.columns)
print(equiv.head())


Index(['GEOID20', 'CA', 'COMMUNIT_1'], dtype='object')
       GEOID20  CA   COMMUNIT_1
0  17031010100   1  Rogers Park
1  17031010201   1  Rogers Park
2  17031010202   1  Rogers Park
3  17031010300   1  Rogers Park
4  17031010400   1  Rogers Park


In [30]:
attributes = {
    "unemployment": "B23025_005E",
    "graduate": "B15003_023E",
    "bachelors": "B15003_022E",
    "snap": "B22003_002E",
    "median_income": "B19013_001E",
    "poverty": "B17001_002E",
    "broadband": "B28002_004E",
    "owner_occupied": "B25003_002E",
    "renter_occupied": "B25003_003E",
    "total_population": "B03002_001E",
    "white": "B03002_003E",
    "black": "B03002_004E",
    "hispanic": "B03002_012E"
}

# tract list with geoid
tract_lists = {
    "Lakeview": [str(tid)[-6:] for tid in equiv[equiv["COMMUNIT_1"] == "Lake View"]["GEOID20"].tolist()],
    "LincolnPark": [str(tid)[-6:] for tid in equiv[equiv["COMMUNIT_1"] == "Lincoln Park"]["GEOID20"].tolist()],
    "NearNorthSide": [str(tid)[-6:] for tid in equiv[equiv["COMMUNIT_1"] == "Near North Side"]["GEOID20"].tolist()],
    "JeffersonPark": [str(tid)[-6:] for tid in equiv[equiv["COMMUNIT_1"] == "Jefferson Park"]["GEOID20"].tolist()]
}

# Populate CSVs

In [31]:
for hood, tract_ids in tract_lists.items():
    os.makedirs(f"ONSA_Data/{hood}", exist_ok=True)

    for attr, code in attributes.items():
        dfs = []
        for year in range(2018, 2024):
            df = fetch_acs(year, [code])
            df = df[df["tract"].isin(tract_ids)]
            df = df[["NAME", code, "state", "county", "tract", "YEAR"]]
            dfs.append(df)

        df_all = pd.concat(dfs, ignore_index=True)
        out_path = f"ONSA_Data/{hood}/{hood}_{attr}_2018_2023.csv"
        df_all.to_csv(out_path, index=False)
        print(f"Saved {out_path}")

Saved ONSA_Data/Lakeview/Lakeview_unemployment_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_graduate_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_bachelors_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_snap_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_median_income_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_poverty_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_broadband_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_owner_occupied_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_renter_occupied_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_total_population_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_white_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_black_2018_2023.csv
Saved ONSA_Data/Lakeview/Lakeview_hispanic_2018_2023.csv
Saved ONSA_Data/LincolnPark/LincolnPark_unemployment_2018_2023.csv
Saved ONSA_Data/LincolnPark/LincolnPark_graduate_2018_2023.csv
Saved ONSA_Data/LincolnPark/LincolnPark_bachelors_2018_2023.csv
Saved ONSA_Data/LincolnPark/LincolnPark_snap

# Calculating geoid + race percentages and majority race

In [32]:
def build_geo_id(row):
    state = str(row['state']).zfill(2)
    county = str(row['county']).zfill(3)
    tract = str(row['tract']).zfill(6)
    return f"1400000US{state}{county}{tract}"

def fetch_race_from_csvs(neighborhood):
    # load pop
    total_df = pd.read_csv(f"ONSA_Data/{neighborhood}/{neighborhood}_total_population_2018_2023.csv")
    total_df['geo_id'] = total_df.apply(build_geo_id, axis=1)

    # load race 
    white_df = pd.read_csv(f"ONSA_Data/{neighborhood}/{neighborhood}_white_2018_2023.csv")
    black_df = pd.read_csv(f"ONSA_Data/{neighborhood}/{neighborhood}_black_2018_2023.csv")
    hisp_df  = pd.read_csv(f"ONSA_Data/{neighborhood}/{neighborhood}_hispanic_2018_2023.csv")

    for df in [white_df, black_df, hisp_df]:
        df['geo_id'] = df.apply(build_geo_id, axis=1)

    merged = total_df.merge(white_df, on=["geo_id","YEAR"], suffixes=("", "_white"))
    merged = merged.merge(black_df, on=["geo_id","YEAR"], suffixes=("", "_black"))
    merged = merged.merge(hisp_df,  on=["geo_id","YEAR"], suffixes=("", "_hisp"))

    # extract cols
    total_col = "B03002_001E"
    white_col = "B03002_003E"
    black_col = "B03002_004E"
    hisp_col  = "B03002_012E"

    for col in [total_col, white_col, black_col, hisp_col]:
        merged[col] = pd.to_numeric(merged[col], errors="coerce")

    # compute %
    merged["white_pct"]    = merged[white_col] / merged[total_col] * 100
    merged["black_pct"]    = merged[black_col] / merged[total_col] * 100
    merged["hispanic_pct"] = merged[hisp_col]  / merged[total_col] * 100

    # majority race
    def majority(row):
        races = {
            "white": row["white_pct"],
            "black": row["black_pct"],
            "hispanic": row["hispanic_pct"]
        }
        return max(races, key=races.get)

    merged["majority_race"] = merged.apply(majority, axis=1)

    return merged[["geo_id","YEAR","white_pct","black_pct","hispanic_pct","majority_race"]]


df_race = fetch_race_from_csvs("LincolnPark")
print(df_race.head())


                 geo_id  YEAR  white_pct  black_pct  hispanic_pct  \
0  1400000US17031071200  2018  73.567194   6.324111      8.201581   
1  1400000US17031070200  2018  80.714452   5.253327      5.603549   
2  1400000US17031070300  2018  85.650350   1.454545      4.391608   
3  1400000US17031832500  2018  84.117150   4.449451      5.744861   
4  1400000US17031071500  2018  84.369332   1.863149      3.248145   

  majority_race  
0         white  
1         white  
2         white  
3         white  
4         white  


# aggregate

In [34]:
neighborhoods = [
    "Irving_Park",
    "Englewood",
    "Portage_Park",
    "South_Lawndale",
    "West_Englewood",
    "LincolnPark",
    "NearNorthSide",
    "JeffersonPark",
    "Lakeview"
    
]

attributes = {
    "unemployment": "B23025_005E",
    "graduate": "B15003_023E",
    "bachelors": "B15003_022E",
    "snap": "B22003_002E",
    "median_income": "B19013_001E",
    "poverty": "B17001_002E",
    "broadband": "B28002_004E",
    "owner_occupied": "B25003_002E",
    "renter_occupied": "B25003_003E",
    "unemployment": "B23025_005E",
    "total_population": "B03002_001E"
}


combined_neighborhoods = []

for hood in neighborhoods:
    dfs = []

    
    for attr, col_code in attributes.items():
        file_path = f"ONSA_Data/{hood}/{hood}_{attr}_2018_2023.csv"
        try:
            df = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"Missing file: {file_path}")
            continue

        df['geo_id'] = df.apply(build_geo_id, axis=1)
        value_col = col_code if col_code in df.columns else df.columns[1]
        df_wide = df.pivot(index="geo_id", columns="YEAR", values=value_col)
        df_wide.columns = [f"{attr}_{year}" for year in df_wide.columns]
        df_wide.reset_index(inplace=True)

        dfs.append(df_wide)

    #load race counts,compute percentages + majority race
    race_dfs = {}
    for race in ['white', 'black', 'hispanic']:
        race_path = f"ONSA_Data/{hood}/{hood}_{race}_2018_2023.csv"
        try:
            race_df = pd.read_csv(race_path)
        except FileNotFoundError:
            print(f"Missing race file: {race_path}")
            continue

        race_df['geo_id'] = race_df.apply(build_geo_id, axis=1)
        # The saved race CSVs hold counts in the 2nd column (e.g., B03002_003E)
        count_col = race_df.columns[1]
        race_wide = race_df.pivot(index="geo_id", columns="YEAR", values=count_col)
        race_wide.columns = [f"{race}_count_{year}" for year in race_wide.columns]
        race_wide.reset_index(inplace=True)
        race_dfs[race] = race_wide

    if race_dfs:
        race_counts = reduce(lambda left, right: pd.merge(left, right, on="geo_id", how="outer"), race_dfs.values())

        # load total population wide
        total_pop_path = f"ONSA_Data/{hood}/{hood}_total_population_2018_2023.csv"
        try:
            total_pop_df = pd.read_csv(total_pop_path)
            total_pop_df['geo_id'] = total_pop_df.apply(build_geo_id, axis=1)
            total_col = "B03002_001E" if "B03002_001E" in total_pop_df.columns else total_pop_df.columns[1]
            total_pop_wide = total_pop_df.pivot(index="geo_id", columns="YEAR", values=total_col)
            total_pop_wide.columns = [f"total_population_{year}" for year in total_pop_wide.columns]
            total_pop_wide.reset_index(inplace=True)
        except FileNotFoundError:
            print(f"Missing total population file: {total_pop_path}")
            total_pop_wide = pd.DataFrame(columns=["geo_id"])

        race_merged = pd.merge(race_counts, total_pop_wide, on="geo_id", how="outer")

        # compute percentages and majority race per year
        for year in range(2018, 2024):
            white_c = f"white_count_{year}"
            black_c = f"black_count_{year}"
            hisp_c  = f"hispanic_count_{year}"
            total_c = f"total_population_{year}"

            race_merged[f"white_pct_{year}"]    = race_merged[white_c] / race_merged[total_c] * 100
            race_merged[f"black_pct_{year}"]    = race_merged[black_c] / race_merged[total_c] * 100
            race_merged[f"hispanic_pct_{year}"] = race_merged[hisp_c]  / race_merged[total_c] * 100

            def majority(row):
                vals = {
                    "white":    row.get(f"white_pct_{year}"),
                    "black":    row.get(f"black_pct_{year}"),
                    "hispanic": row.get(f"hispanic_pct_{year}")
                }
                vals = {k: (-1 if pd.isna(v) else v) for k, v in vals.items()}
                return max(vals, key=vals.get)

            race_merged[f"majority_race_{year}"] = race_merged.apply(majority, axis=1)

        # Keep only pct and majority col
        keep_cols = ["geo_id"] + [c for c in race_merged.columns if c.startswith(("white_pct_", "black_pct_", "hispanic_pct_", "majority_race_"))]
        race_merged = race_merged[keep_cols]

        dfs.append(race_merged)  

    # merge all
    if dfs:
        merged = reduce(lambda left, right: pd.merge(left, right, on="geo_id", how="outer"), dfs)
        merged['neighborhood'] = hood
        combined_neighborhoods.append(merged)


final_df = pd.concat(combined_neighborhoods, ignore_index=True)
final_df = final_df.loc[:, ~final_df.columns.str.endswith('_y')]
final_df.columns = final_df.columns.str.replace('_x', '', regex=False)


print(final_df.head())
#final_df.to_csv("agg_data.csv", index=False)

                 geo_id  unemployment_2018  unemployment_2019  \
0  1400000US17031160100                105                109   
1  1400000US17031160200                 46                 57   
2  1400000US17031160300                113                 60   
3  1400000US17031160400                 85                 76   
4  1400000US17031160501                 78                109   

   unemployment_2020  unemployment_2021  unemployment_2022  unemployment_2023  \
0                 96                 83                 54                 37   
1                123                104                 94                 62   
2                175                206                220                262   
3                184                214                203                205   
4                171                179                183                166   

   graduate_2018  graduate_2019  graduate_2020  ...  majority_race_2021  \
0            240            265            335 